# Strength
<strong>J. M. Maas @ IfB, ETH Zürich, v. 2026-01-19</strong>

<div class='alert alert-info'>
    Select ▸▸ <strong>Restart the kernel and run all cells</strong> to run the widgets in this notebook and explore the strength data.
</div>

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib widget
from datetime import datetime

# Import required modules
import utils.scleronomic_utils as scutils

The next cell imports the data points of the strength data.

In [ ]:
# Import strength data
df = pd.read_csv('data/02_strength/strength_data.csv', index_col=['sheet', 'sample'])
display(df)

Run the next cell to expore the strength data.

In [ ]:
# Define reference points
styles_p = [
    dict(s=60, marker='d', color='cornflowerblue', ec='black'),
    dict(s=60, marker='d', color='chocolate', ec='black')
]
ref_s_points = {
    'niemz': dict(use=True, mc=12, ftl=87.2, ftr=3.96, ftt=3.07, fcl=40.2, fcr=4.1, fct=4.2),  # internal ETH measurements reported in Niemz et al. (2023), p. 468
    'dahl': dict(use=True, mc=12, fsrl=6.06, fslr=6.21, fstl=4.17, fslt=4.54, fstr=1.65, fsrt=1.64),  # Dahl and Malo (2009), ultimate stresses on Arcan samples of Norway spruce
    'schmidt': dict(use=False, mc=12, ftl=65.5, ftr=3.75, ftt=2.79, fcl=50.3, fcr=6.0, fct=6.0, fsrt=1.83, fstl=5.34, fsrl=6.34)  # Schmidt and Kaliske (2006), which determined an average from literature where they do not give the reference
}

# Define piecewise linear function
multilinear = lambda MC, x, y: sp.interpolate.interp1d(x, y, kind='linear', bounds_error=False, fill_value=np.nan)(MC)

# Define reference lines
styles_l = [
    dict(ls='--', color='#777777'),
    dict(ls=':' , color='#777777')
]
ref_s_lines = {  # MC always in [%]
    'akter': dict(use=False, fcr=lambda MC: 0.0005*MC**3 - 0.02524*MC**2 + 0.1604*MC + 6.24),  # Akter et al. (2023) for offset strain 1%
    'hassani': dict(  # Hassani et al. (2015), constructed from Schmidt and Kaliske (2006) and Saft and Kaliske (2011), with MC-dependence scaled by strength relationships at 12% MC
        use=True,
        fcr=lambda MC: -0.270*MC + 9.24,
        fct=lambda MC: -0.270*MC + 9.24,
        fcl=lambda MC: -2.150*MC + 68.8,
        ftl=lambda MC: -3.275*MC + 104.8,
        fsrt=lambda MC: -0.063*MC + 2.585,
        fsrl=lambda MC: -0.227*MC + 9.064,
        fstl=lambda MC: -0.178*MC + 7.477
    ),
    'gerhards': dict(  # Gerhards (1982), trends derived from multiple hardwood and softwood species, multiplied with ref_s_points
        use=False,
        fcr=lambda MC: multilinear(MC, [6, 12, 20], [1.3, 1.0, 0.7])*ref_s_points['niemz']['fcr'],
        fct=lambda MC: multilinear(MC, [6, 12, 20], [1.3, 1.0, 0.7])*ref_s_points['niemz']['fct'],
        fcl=lambda MC: multilinear(MC, [6, 12, 20], [1.35, 1.0, 0.65])*ref_s_points['niemz']['fcl'],
        ftr=lambda MC: multilinear(MC, [6, 12, 20], [1.12, 1.0, 0.8])*ref_s_points['niemz']['ftr'],
        ftt=lambda MC: multilinear(MC, [6, 12, 20], [1.12, 1.0, 0.8])*ref_s_points['niemz']['ftt'],
        ftl=lambda MC: multilinear(MC, [6, 12, 20], [1.08, 1.0, 0.85])*ref_s_points['niemz']['ftl'],
        fslr=lambda MC: multilinear(MC, [6, 12, 20], [1.18, 1.0, 0.82])*ref_s_points['dahl']['fslr'],
        fslt=lambda MC: multilinear(MC, [6, 12, 20], [1.18, 1.0, 0.82])*ref_s_points['dahl']['fslt']
    )
}

In [ ]:
# Create widget
df_export = [pd.DataFrame()]

# Define fitting function
def fit_strength(df):
    '''
    Fit a linear relationship into the data points of the given dataframe.
    Input:
        df  (pd.DataFrame) : DataFrame with column 'mc' as fitted x-coordinates and column 'mc_stress' and fitted y-coordinates.
    Output:
        fit (dict)         : Solution of fit. The keys are:
                             fun : Fitted function.
                             p : Best fitted parameter.
                             success : Success of fitting
                             obj : Fitting model object.
        R2  (float)        : The fit's coefficient of determination.
    '''
    # Initialize
    fit, R2 = None, 0.0

    # Extract data
    df = df.dropna(subset=['mc', 'max_stress'])
    mc = df['mc'].to_numpy() * 100
    max_stress = df['max_stress'].to_numpy()

    # Define minimization
    f = lambda mc, p: np.poly1d(p)(mc)
    p0 = [-0.001, np.max(max_stress)]
    sol = scutils.lsq_fitting(f, xdata=mc, ydata=max_stress, p0=p0, method='lmfit', options=dict(maxiter=2000))
    p, success = sol['x'], sol['success']
    
    if not success:
        label = ', '.join(df['label'].unique())
        print(f'Warning: No fitting success for {label}: {sol.message}')

    # Return results
    fit = {
        'fun': f,
        'p':  p,
        'success': success,
        'err': sol['err'],
        'obj': sol['obj']
    }
    R2 = sol['R2']
    
    return fit, R2


# Define plotting function
def plot_strength_distribution(df, direction, ax, fit_plot=True, show_confidence=True, sigma=1, show_reference=True, offsets=['cR']):
    '''
    Plot the strength distribution of a given direction.
    Input:
        df        (pd.DataFrame) : Parameter dataframe that contains the index levels 'sheet' and 'samples'
                                   and the columns 'mc' and 'max_stress'.
        direction (str)          : Direction of the plotted data. Options: 'L', 'R', 'T' for uniaxial data,
                                   'RT', 'RL', 'TL' for shear data.
        ax        (mpl.axes)     : Axis to draw the plot into.
        sigma     (int)          : Optional. Sigma for plotting the confidence bands and prediction bands.
        offsets   (list)         : List where offset stress instead of ultimate stress should be used. Options:
                                   'cR', 'cT', 'cL', 'tR', 'tT', 'tL', 'sRT', 'sTR', 'sLR', 'sRL', 'sTL',
                                   'sLT' where lower letters indicate load type and upper letters direction.
                                
    Output:
        df_p      (pd.DataFrame) : Stored parameters of the fit.
    '''
    # Initialize
    if len(direction) == 1:
        loading_types = ['compression', 'tension']
        directions = [direction, direction]
    else:
        loading_types = ['shear', 'shear']
        directions = [direction, direction[::-1]]
    
    # Setup formatting
    colors = ['cornflowerblue', 'chocolate']
    markers = ['o', '^']
    fcs = [(1.0, 1.0, 1.0, 0.0), (1.0, 1.0, 1.0, 0.0)]
                                 
    p2data = lambda fit: dict( [(f'p_{i}', p_i) for i, p_i in enumerate(fit['p']) ] + [(f'Std_p_{i}', err_i) for i, err_i in enumerate(fit['err']) ] )
    
    # Iterate over all loading types
    lgd_labels, lgd_symbols, R2s, dfs = [], [], [], []
    dfs_export = []
    for i, (loading_type, direction) in enumerate(zip(loading_types, directions)):
        # Select data
        data = df.query(f'loading_type == "{loading_type}" and direction == "{direction}"')

        # Set offsets
        case = loading_type[0] + direction
        if case in offsets:  # check whether in offsets list
            data = data.copy().loc[ ~np.isnan(data['offset_stress']) ]
            data['max_stress'] = data['offset_stress']
        
        # Plot data
        mc, max_stress = data['mc']*100, data['max_stress']
        ax.scatter(mc, max_stress, zorder=3, marker=markers[i], color=colors[i], fc=fcs[i])
        
        # Format axis
        ax.set_xlim([0, 25])
        ax.grid(True)
        
        # Create fit
        fit, R2 = fit_strength(data)
        R2s.append( R2 )
        fit_mc = np.linspace(0, 25, 100)
        fit_max_stress = fit['fun'](fit_mc, fit['p'])

        if fit_plot:
            # Plot fit
            ax.plot(fit_mc, fit_max_stress, zorder=4, c=colors[i])
    
            if show_confidence:
                # Plot error
                model = fit['obj']
                fit_confidence = model.eval_uncertainty(x=fit_mc, sigma=sigma)
                fit_prediction = model.dely_predicted
                ax.fill_between(fit_mc, fit_max_stress-fit_confidence, fit_max_stress+fit_confidence, zorder=2, color=colors[i], ec='none', alpha=0.5)
                ax.fill_between(fit_mc, fit_max_stress-fit_prediction, fit_max_stress+fit_prediction, zorder=1, color=colors[i], ec='none', alpha=0.3)
            
        # Create legend items
        lgd_labels.append( fr'$f_{{ {loading_type[0]}, {direction} }}$ ($R^2 = {R2:.2f}$)' )
        lgd_symbols.append( plt.Line2D([0], [0], color=colors[i], lw=2, linestyle='-', marker=markers[i], mfc=fcs[i], mec=colors[i]) )
        

        if show_reference:
            # Add reference points
            for key in ref_s_points.keys():
                ref = ref_s_points[key]
                if not ref['use']:  # check whether reference is set for usage
                    continue
                case_key = f'f{case.lower()}'
                if case_key in list( ref.keys() ):  # is loading case in reference?
                    mc_ref, f_ref = ref['mc'], ref[case_key]
                    ax.scatter(mc_ref, f_ref, zorder=7, **styles_p[i])
    
            # Add reference lines
            for key in ref_s_lines.keys():
                ref = ref_s_lines[key]
                if not ref['use']:  # check whether reference is set for usage
                    continue
                case_key = f'f{case.lower()}'
                if case_key in list( ref.keys() ):  # is loading case in reference?
                    f_ref = ref[case_key](fit_mc)
                    ax.plot(fit_mc, f_ref, zorder=6, **styles_l[i])
                
        
        # Store data
        dfs.append( pd.DataFrame( data=p2data(fit), index=pd.Index([f'f_{loading_type[0]}{direction}'], name='constant') ) )
        dfs_export.append( data )

    # Add average
    data = df.query(f'loading_type in {loading_types} and direction in {directions}')
    fit, R2 = fit_strength(data)
    R2s.append( R2 )
    dfs.append( pd.DataFrame( data=p2data(fit), index=pd.Index([f'f_avg{direction}'], name='constant') ) )
    fit_mc = np.linspace(0, 25, 100)
    fit_max_stress = fit['fun'](fit_mc, fit['p'])
    if fit_plot:
        ax.plot(fit_mc, fit_max_stress, zorder=5, c='#454545', ls='-.')
    
    # Format plot
    ax.legend(lgd_symbols, lgd_labels, loc='upper right', fontsize='small')
    
    # Assemble data
    df_p = pd.concat(dfs)
    df_p['R2'] = R2s

    df_export[0] = pd.concat(dfs_export)
    
    return df_p


# Export function
def export_csv(button):
    if len(df_export) > 0:
        export_path = f'export_strength_{w_direction.value}.csv'
        df_export[0].to_csv(export_path)
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with w.children[-1]:
            w.children[-1].clear_output()
            print(f'({timestamp}) Successfully saved data to {export_path}.')
button = widgets.Button(description='Export CSV', button_style='primary', icon='save')
button.on_click(export_csv)
display(button)


# Widget function
def visualize_strength(df, direction, ax, fit_plot=True, show_confidence=True, sigma=1, show_reference=True):
    # Initialize
    ax.cla()

    # Generate plot
    params = plot_strength_distribution(df, direction, ax, fit_plot, show_confidence, sigma, show_reference)
    display(params)

    # Format labels
    ax.set_xlabel('Moisture Content [%]')
    ax.set_ylabel('Strength [MPa]')
    ax.set_title(f'Selection direction: {direction}')
    

# Create widget
fig, ax = plt.subplots()
w_direction = widgets.Dropdown(options=df['direction'].unique(), description='direction')
w = widgets.interactive(
    visualize_strength,
    df=widgets.fixed(df),
    direction=w_direction,
    ax=widgets.fixed(ax),
    sigma=widgets.FloatSlider(value=1, min=0, max=3)
)
display(w)
# fit_plot=True, show_confidence=True, sigma=1, show_reference=True

Use the control elements to explore the data.